In [ ]:
estados = set(["q0", "q1", "q2", "q3"])
alfabeto = set(['0', '1', '2'])
funcao_de_transicao = {
    ('q0', '0'): 'q0',
    ('q0', '1'): 'q1',
    ('q0', '2'): 'q3',
    ('q1', '0'): 'q3',
    ('q1', '1'): 'q1',
    ('q1', '2'): 'q2',
    ('q2', '0'): 'q3',
    ('q2', '1'): 'q3',
    ('q2', '2'): 'q2',
    ('q3', '0'): 'q3',
    ('q3', '1'): 'q3',
    ('q3', '2'): 'q3',
}
estado_inicial = 'q0'
estados_finais = set(['q1', 'q2'])

In [ ]:
print(automato_1.maquina_estados.reconhecer_cadeia('0001112'))

In [ ]:
from collections import deque

def escrever_fita(fita: deque, cadeia: str) -> None:
    """Escreve uma cadeia de caracteres na fita."""
    fita.clear()
    fita = deque(cadeia)
    fita.append("\0")
    return fita

fita = deque()
fita = escrever_fita(fita, "0001112")
fita.append("\0")
print(fita)

In [ ]:
from itertools import accumulate
from collections import deque
from functools import reduce

def muda_estado(estado_atual, simbolo) -> str:
    estado_atual = funcao_de_transicao2[estado_atual][simbolo]
    return estado_atual

estado_atual = "q0"
estado_atual = muda_estado(estado_atual, "2")
print(estado_atual)

In [ ]:
cadeia = deque("0001112")
transicoes = accumulate(cadeia, muda_estado, initial='q0')
transicoes2 = reduce(muda_estado, cadeia, 'q0')

#### Conversão de RegEx em AFND
O autômato finito não-determinístico que aceita a linguagem definida por r pode ser obtido através da aplicação do algoritmo a seguir que especifica as regras de mapeamento parciais que abrangem casos triviais de sentenças (itens 1, 2 e 3) e cada um dos operadores de união (4), concatenação (5) e fechamento (6)

Obs:. F são finais de segmento
```
Casos triviais:
1. ->   q0: {
        ε: q1
        }
        Q = {q0, q1}
        F = {q1}

2. ->   q0: {
        NULL:q0
        }
        Q = {q0, q1}
        F = {q1}

3. ->   q0: {
        σ: q1
        }
        Q = {q0, q1}
        F = {q1}

Operadores:
4. a|b = {
    q0: {
        ε: q1,
        ε: q2,
    },
    q1: {
        a: q2
    },
    q3: {
        b: q4
    },
    q4: {
        ε: q5
    },
    q2: {
        ε: q5
    }
}
    Q = {q0, q1, q2, q3, q4, q5}
    F = {q5}

5. ab = {
    q0: {
        a: q1
    },
    q1: {
        ε: q2
    },
    q2: {
        b: q3
    },
    q3: {
        ε: q4
    }
}
    Q = {q0, q1, q2, q3, q4}
    F = {q4}

6. a* = {
    q0: {
       ε: q1,
       ε: q5, 
    }
    q1: {
        ε: q2
    },
    q2: {
        ε: q3
    },
    q3: {
        ε: q4
    },
    q4: {
        ε: q5,
        ε: q1
    }
}
    Q = {q0, q1, q2, q3, q4, q5}
    F = {q5}
```

passo-a-passo: 
1. encontrar os parenteses e resolver as expressões.
2. aplicar a funcao a cada membro da expressão regular.


#### Implementação usando namedtuples (mais barata)

In [ ]:
from collections import namedtuple
import pandas as pd


def construir_transicoes(alfabeto: str, epsilon=False) -> namedtuple:
    """Constrói uma namedtuple com o alfabeto da linguagem e se o
    automato aceita transições vazias.
        Args:
            alfabeto (str) -- Alfabeto da linguagem.
            epsilon (bool, optional) -- Se o automato aceita transições 
                                        vazias. Defaults to False.
        Returns:
            namedtuple: Tupla com o alfabeto e se aceita transições vazias.
    """
    fields = f'{" ".join(alfabeto) + (" epsilon" if epsilon else "")}'
    alfabeto_tuple = namedtuple('transita', fields)
    return alfabeto_tuple

def construir_bloco_or(a: str, b: str, n: int) -> namedtuple:
    """Constrói um bloco de transições para o operador OR de acordo com
    o algoritmo de Thompson.
        Args:
            a (str) -- Primeiro operando.
            b (str) -- Segundo operando.
            n (int) -- Número do bloco.
        Returns:
            namedtuple: Tupla com os estados do bloco.
    """
    transicoes = construir_transicoes(f"{a}{b}", epsilon=True)
    bloco_or = namedtuple('estado', f'q{n}0 q{n}1 q{n}2 q{n}3 q{n}4')

    q0 = transicoes('','',(f'q{n}1', f'q{n}2'))
    q1 = transicoes(f'q{n}2','','')
    q2 = transicoes('','',f'q{n}5')
    q3 = transicoes('',f'q{n}4','')
    q4 = transicoes('','',f'q{n}5')

    return bloco_or(q0, q1, q2, q3, q4)

def construir_fecho(simbolo: str) -> namedtuple:
    """Constrói um bloco de transições para o operador de fecho de Kleene
    de acordo com o algoritmo de Thompson.
        Args:
            simbolo (str) -- operando do operador de fecho de Kleene.
        Returns:
            namedtuple: Tupla com os estados do bloco.
    """
    pass

bloco_teste_2 = construir_bloco_or('c', 'd', 0)
bloco_teste_2_df = pd.DataFrame(bloco_teste_2, index=bloco_teste_2._fields)
bloco_teste_2_df

In [ ]:
bloco_or_dict = {
    'q00': {
        'c': '',
        'd': '',
        'epsilon': ('q01', 'q02')
        },
    'q01': {
        'c': 'q02',
        'd': '',
        'epsilon': ''
        },
    'q02': {
        'c': '',
        'd': '',
        'epsilon': 'q05'
        },
    'q03': {
        'c': '',
        'd': 'q04',
        'epsilon': ''
        },
    'q04': {
        'c': '',
        'd': '',
        'epsilon': 'q05'}
    }
bloco_teste_3_df = pd.DataFrame(bloco_or_dict).T
bloco_teste_3_df.loc['q00'].epsilon

In [ ]:
nao_determinismo = any(isinstance(simbolo, tuple) for estado in bloco_teste_2\
                    for simbolo in estado)
print(nao_determinismo)

In [ ]:
class Transdutor(Automato):
    pass
        
automato_2 = Transdutor(estados, alfabeto, funcao_de_transicao2, estado_inicial, estados_finais)

In [ ]:
automato_2.estados_finitos
print(automato_2.maquina_estados.reconhecer_cadeia('00000'))
print(isinstance(automato_2, Automato))

In [ ]:
# # Breadth First Search (BFS)

# automato_2.maquina_estados.estado_atual = automato_2.estado_inicial
# # print(consumidos)
# def bfs(automato: Automato, cadeia: str):
#     grafo = automato.transicoes
#     print(grafo)
#     consumidos = deque([])
#     transitados = deque([])
#     automato.escrever_fita(cadeia)
#     while automato.fita and cadeia not in consumidos:
#         alvo = automato.fita.popleft()
#         vertice = automato.maquina_estados.estado_atual
#         if cadeia != "".join(consumidos) and alvo in grafo[vertice].keys():
#             consumidos.append(alvo)
#             print(consumidos)
#             transitados.append(vertice)
#             print(transitados)
#             automato.maquina_estados.estado_atual = grafo[vertice][alvo]
#     else:
#             consumidos.append(alvo)
#             print(consumidos)
#             transitados.append(vertice)
#             print(transitados)
#     print(transitados)
#     return consumidos

# print(bfs(automato_2, ''))

In [32]:
# def reconhecer_cadeia(self, cadeia: str):
#     self.maquina_estados.resetar()    
#     self.escrever_fita(cadeia)
#     self.maquina_estados.gravar_movimento(self.fita)
#     while self.fita:
#         simbolo = self.fita.popleft()
#         try:
#             self.maquina_estados.config_seguinte(simbolo)
#             self.maquina_estados.gravar_movimento(self.fita)
#         except KeyError:
#             return False
#     estado_parada = self.maquina_estados.estado_atual
#     return bool(estado_parada in self.estados_finais)

# reconhecer_cadeia(self_2, '011')
# print(self_2.maquina_estados.movimentos)
# print(self_2.maquina_estados.apresentar_movimentos())

In [39]:
def reconhecer_cadeia(automato: Automato, cadeia: str):
    automato.maquina_estados.resetar()
    automato.escrever_fita(cadeia)    
    ler, fita = automato.maquina_estados.config_seguinte, automato.fita
    for _ in (ler(celula) for celula in fita):        
        automato.maquina_estados.movimentos.append(automato.maquina_estados.estado_atual)
    estado_parada = automato.maquina_estados.estado_atual
    return bool(estado_parada in automato.estados_finais)

reconhecer_cadeia(automato_2, '011')
print(list(automato_2.maquina_estados.movimentos))

['q0', 'q1', 'q1']


In [ ]:
grafo['q0'].keys()

dict_keys(['0', '1', '2'])

In [29]:
diacriticos = set(['é', 'í', 'á', 'ó', 'ú', 'õ', 'ã', 'â', 'ê', 'ô', 'ç', 'à'])
alfabeto_entrada = set(set([chr(i) for i in range(32,128)]).union(diacriticos).union(set(map(str.upper, diacriticos))))

for i in alfabeto_entrada:
    if i != ' ' and i != 'C' and i != 'c':
        print("('q13', '" + i + "'): 'q1',")

('q13', 'S'): 'q1',
('q13', 'ç'): 'q1',
('q13', ','): 'q1',
('q13', 'N'): 'q1',
('q13', 'm'): 'q1',
('q13', '&'): 'q1',
('q13', 'Í'): 'q1',
('q13', 'Ê'): 'q1',
('q13', 'D'): 'q1',
('q13', 'B'): 'q1',
('q13', 'à'): 'q1',
('q13', 'ô'): 'q1',
('q13', '"'): 'q1',
('q13', '{'): 'q1',
('q13', '%'): 'q1',
('q13', 'á'): 'q1',
('q13', 'À'): 'q1',
('q13', 'E'): 'q1',
('q13', '?'): 'q1',
('q13', 'Z'): 'q1',
('q13', 'x'): 'q1',
('q13', '7'): 'q1',
('q13', '4'): 'q1',
('q13', ']'): 'q1',
('q13', ';'): 'q1',
('q13', 'r'): 'q1',
('q13', '/'): 'q1',
('q13', '!'): 'q1',
('q13', 'P'): 'q1',
('q13', '='): 'q1',
('q13', 'í'): 'q1',
('q13', 'i'): 'q1',
('q13', 'ã'): 'q1',
('q13', 'u'): 'q1',
('q13', 'V'): 'q1',
('q13', '.'): 'q1',
('q13', '1'): 'q1',
('q13', '''): 'q1',
('q13', 'v'): 'q1',
('q13', '2'): 'q1',
('q13', 'Y'): 'q1',
('q13', '#'): 'q1',
('q13', 'n'): 'q1',
('q13', ')'): 'q1',
('q13', 'w'): 'q1',
('q13', '>'): 'q1',
('q13', '<'): 'q1',
('q13', '_'): 'q1',
('q13', 'g'): 'q1',
('q13', 'J'): 'q1',
